In [6]:
path='C:/Users/student/Downloads/데이터들/'

In [48]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.optimizers import *
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer

from nltk.tokenize import sent_tokenize
from konlpy.corpus import kolaw
from konlpy.tag import Okt

In [32]:
chatbotData=pd.read_csv(path+'ChatData.csv')
question, answer = list(chatbotData['Q']), list(chatbotData['A'])

NameError: name 'path' is not defined

In [30]:
len(answer)

NameError: name 'answer' is not defined

In [33]:
c=kolaw.open('constitution.txt').read()

In [34]:
sent_tokenize(c) # 문장단위로 

['대한민국헌법\n\n유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여 정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확보할 것을 다짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회의 의결을 거쳐 국민투표에 의하여 개정한다.',
 '제1장 총강\n  제1조 ① 대한민국은 민주공화국이다.',
 '②대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다.',
 '제2조 ① 대한민국의 국민이 되는 요건은 법률로 정한다.',
 '②국가는 법률이 정하는 바에 의하여 재외국민을 보호할 의무를 진다.',
 '제3조 대한민국의 영토는 한반도와 그 부속도서로 한다.',
 '제4조 대한민국은 통일을 지향하며, 자유민주적 기본질서에 입각한 평화적 통일 정책을 수립하고 이를 추진한다.',
 '제5조 ① 대한민국은 국제평화의 유지에 노력하고 침략적 전쟁을 부인한다.',
 '②국군은 국가의 안전보장과 국토방위의 신성한 의무를 수행함을 사명으로 하며, 그 정치적 중립성은 준수된다.',
 '제6조 ① 헌법에 의하여 체결·공포된 조약과 일반적으로 승인된 국제법규는 국내법과 같은 효력을 가진다.',
 '②외국인은 국제법과 조약이 정하는 바에 의하여 그 지위가 보장된다.',
 '제7조 ① 공무원은 국민전체에 대한 봉사자이며, 국민에 대하여 책임을 진다.',
 '②공무원의 신분과 정치적 중립성은 법률이 정하는 바에 의하여 

In [35]:
okt=Okt()
okt.pos("오늘은 날씨가 따뜻합니다")

[('오늘', 'Noun'),
 ('은', 'Josa'),
 ('날씨', 'Noun'),
 ('가', 'Josa'),
 ('따뜻합니다', 'Adjective')]

In [36]:
doc0=[" ".join([''.join(w) for w, t in okt.pos(s) if t not in ['Number','Foreign'] and w not in ['제','조']]) for s in sent_tokenize(c)]

In [37]:
tokenizer=Tokenizer()
tokenizer.word_index

{}

In [38]:
tokenizer.fit_on_texts(doc0)
doc=[tok for tok in tokenizer.texts_to_sequences(doc0) if len(tok)>1]

In [39]:
maxLen=max([len(x)-1 for x in doc])
print(maxLen)
vocabSize=len(tokenizer.word_index)+1
print(vocabSize)

187
1165


In [11]:
sent=[s for s in sent_tokenize(c)]
sent[0]

'대한민국헌법\n\n유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여 정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확보할 것을 다짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회의 의결을 거쳐 국민투표에 의하여 개정한다.'

In [12]:
sent_tokenize(c)

['대한민국헌법\n\n유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여 정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확보할 것을 다짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회의 의결을 거쳐 국민투표에 의하여 개정한다.',
 '제1장 총강\n  제1조 ① 대한민국은 민주공화국이다.',
 '②대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다.',
 '제2조 ① 대한민국의 국민이 되는 요건은 법률로 정한다.',
 '②국가는 법률이 정하는 바에 의하여 재외국민을 보호할 의무를 진다.',
 '제3조 대한민국의 영토는 한반도와 그 부속도서로 한다.',
 '제4조 대한민국은 통일을 지향하며, 자유민주적 기본질서에 입각한 평화적 통일 정책을 수립하고 이를 추진한다.',
 '제5조 ① 대한민국은 국제평화의 유지에 노력하고 침략적 전쟁을 부인한다.',
 '②국군은 국가의 안전보장과 국토방위의 신성한 의무를 수행함을 사명으로 하며, 그 정치적 중립성은 준수된다.',
 '제6조 ① 헌법에 의하여 체결·공포된 조약과 일반적으로 승인된 국제법규는 국내법과 같은 효력을 가진다.',
 '②외국인은 국제법과 조약이 정하는 바에 의하여 그 지위가 보장된다.',
 '제7조 ① 공무원은 국민전체에 대한 봉사자이며, 국민에 대하여 책임을 진다.',
 '②공무원의 신분과 정치적 중립성은 법률이 정하는 바에 의하여 

In [15]:
# konlpy.rog => API => Okt
temp=sent_tokenize(c)[0:5]
for i in range(10):
    for w,t in okt.pos(temp[i]): # 형태소로 분리하는 함수
        print('w:',w)
        print("t:",t)

w: 대한민국
t: Noun
w: 헌법
t: Noun
w: 


t: Foreign
w: 유구
t: Noun
w: 한
t: Josa
w: 역사
t: Noun
w: 와
t: Josa
w: 전통
t: Noun
w: 에
t: Josa
w: 빛나는
t: Verb
w: 우리
t: Noun
w: 대
t: Modifier
w: 한
t: Modifier
w: 국민
t: Noun
w: 은
t: Josa
w: 3
t: Number
w: ·
t: Punctuation
w: 1
t: Number
w: 운동
t: Noun
w: 으로
t: Josa
w: 건립
t: Noun
w: 된
t: Verb
w: 대한민국
t: Noun
w: 임시정부
t: Noun
w: 의
t: Josa
w: 법
t: Noun
w: 통과
t: Noun
w: 불의
t: Noun
w: 에
t: Josa
w: 항거
t: Noun
w: 한
t: Josa
w: 4
t: Number
w: ·
t: Punctuation
w: 19
t: Number
w: 민주
t: Noun
w: 이념
t: Noun
w: 을
t: Josa
w: 계승
t: Noun
w: 하고
t: Josa
w: ,
t: Punctuation
w: 조국
t: Noun
w: 의
t: Josa
w: 민주
t: Noun
w: 개혁
t: Noun
w: 과
t: Josa
w: 평화
t: Noun
w: 적
t: Suffix
w: 통일
t: Noun
w: 의
t: Josa
w: 사명
t: Noun
w: 에
t: Josa
w: 입
t: Noun
w: 각하
t: Noun
w: 여
t: Josa
w: 정의
t: Noun
w: ·
t: Punctuation
w: 인도
t: Noun
w: 와
t: Josa
w: 동포
t: Noun
w: 애
t: Noun
w: 로써
t: Noun
w: 민족
t: Noun
w: 의
t: Josa
w: 단결
t: Noun
w: 을
t: Josa
w: 공고
t: Noun
w: 히
t: Adverb
w: 하고
t: Verb
w: ,
t: Punctuation
w

IndexError: list index out of range

In [13]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(doc0)

In [17]:
len(tokenizer.index_word)
tokenizer.word_index

{'의': 1,
 '에': 2,
 '을': 3,
 '은': 4,
 '이': 5,
 '한다': 6,
 '·': 7,
 '를': 8,
 '법률': 9,
 '는': 10,
 '한': 11,
 '할': 12,
 '정': 13,
 '수': 14,
 '대통령': 15,
 '과': 16,
 '국가': 17,
 '또는': 18,
 '국회': 19,
 '의하여': 20,
 '하는': 21,
 '국민': 22,
 '하여': 23,
 '로': 24,
 '관': 25,
 '있다': 26,
 '때': 27,
 '헌법': 28,
 '그': 29,
 '에는': 30,
 '가': 31,
 '으로': 32,
 '와': 33,
 '모든': 34,
 '바': 35,
 '위': 36,
 '적': 37,
 '하며': 38,
 '하여야': 39,
 '된': 40,
 '아니': 41,
 '기타': 42,
 '가진다': 43,
 '및': 44,
 '장': 45,
 '사항': 46,
 '인': 47,
 '에서': 48,
 '권리': 49,
 '안': 50,
 '된다': 51,
 '필요한': 52,
 '없다': 53,
 '의원': 54,
 '자유': 55,
 '정부': 56,
 '선거': 57,
 '임명': 58,
 '하고': 59,
 '되지': 60,
 '직무': 61,
 '국무총리': 62,
 '경제': 63,
 '의무': 64,
 '조직': 65,
 '자': 66,
 '하': 67,
 '국회의원': 68,
 '임기': 69,
 '의결': 70,
 '공무원': 71,
 '경우': 72,
 '이상': 73,
 '국무위원': 74,
 '대법원': 75,
 '하기': 76,
 '범위': 77,
 '이를': 78,
 '다': 79,
 '될': 80,
 '받지': 81,
 '법관': 82,
 '보호': 83,
 '정책': 84,
 '정당': 85,
 '헌법재판소': 86,
 '재적': 87,
 '회의': 88,
 '항의': 89,
 '관리': 90,
 '진다': 91,
 '노력': 92,
 '보장': 93,
 

In [18]:
tokenizer.texts_to_sequences(doc0)

[[102,
  28,
  602,
  11,
  603,
  33,
  604,
  2,
  605,
  323,
  606,
  11,
  22,
  4,
  7,
  427,
  32,
  607,
  40,
  102,
  608,
  1,
  324,
  609,
  428,
  2,
  610,
  11,
  7,
  177,
  611,
  3,
  429,
  59,
  325,
  1,
  177,
  612,
  16,
  178,
  37,
  179,
  1,
  430,
  2,
  613,
  614,
  110,
  615,
  7,
  616,
  33,
  617,
  618,
  215,
  326,
  1,
  619,
  3,
  327,
  620,
  59,
  34,
  136,
  37,
  621,
  16,
  428,
  8,
  622,
  38,
  431,
  16,
  432,
  8,
  623,
  32,
  433,
  37,
  137,
  259,
  138,
  8,
  624,
  625,
  23,
  122,
  7,
  63,
  7,
  136,
  7,
  260,
  1,
  34,
  261,
  2,
  123,
  626,
  1,
  328,
  8,
  627,
  59,
  329,
  3,
  434,
  628,
  629,
  262,
  38,
  55,
  33,
  49,
  2,
  630,
  216,
  16,
  64,
  8,
  631,
  262,
  23,
  50,
  435,
  22,
  139,
  1,
  436,
  217,
  3,
  632,
  263,
  633,
  435,
  634,
  37,
  47,
  635,
  178,
  33,
  636,
  637,
  2,
  638,
  218,
  264,
  323,
  437,
  16,
  323,
  437,
  1,
  639,
  1,
  640,
  16,
 

In [19]:
np.shape(tokenizer.texts_to_sequences(doc0))

(357,)

In [25]:
# for a in tokenizer.texts_to_sequences(doc0):
#     if len(a)<=1:
#         print(len(a))
doc=[tok for tok in tokenizer.texts_to_sequences(doc0) if len(tok)>1]     


In [40]:
#maxLen=max([len(x) for x in doc])
#tokenizer.word_counts    
vocabSize=len(tokenizer.word_index)+1
print(maxLen, vocabSize) # 187, 1165

187 1165


In [41]:
doc

[[102,
  28,
  602,
  11,
  603,
  33,
  604,
  2,
  605,
  323,
  606,
  11,
  22,
  4,
  7,
  427,
  32,
  607,
  40,
  102,
  608,
  1,
  324,
  609,
  428,
  2,
  610,
  11,
  7,
  177,
  611,
  3,
  429,
  59,
  325,
  1,
  177,
  612,
  16,
  178,
  37,
  179,
  1,
  430,
  2,
  613,
  614,
  110,
  615,
  7,
  616,
  33,
  617,
  618,
  215,
  326,
  1,
  619,
  3,
  327,
  620,
  59,
  34,
  136,
  37,
  621,
  16,
  428,
  8,
  622,
  38,
  431,
  16,
  432,
  8,
  623,
  32,
  433,
  37,
  137,
  259,
  138,
  8,
  624,
  625,
  23,
  122,
  7,
  63,
  7,
  136,
  7,
  260,
  1,
  34,
  261,
  2,
  123,
  626,
  1,
  328,
  8,
  627,
  59,
  329,
  3,
  434,
  628,
  629,
  262,
  38,
  55,
  33,
  49,
  2,
  630,
  216,
  16,
  64,
  8,
  631,
  262,
  23,
  50,
  435,
  22,
  139,
  1,
  436,
  217,
  3,
  632,
  263,
  633,
  435,
  634,
  37,
  47,
  635,
  178,
  33,
  636,
  637,
  2,
  638,
  218,
  264,
  323,
  437,
  16,
  323,
  437,
  1,
  639,
  1,
  640,
  16,
 

In [49]:
def genData(doc, maxLen, vocabSize):
    for sent in doc:
        inputs=[]
        targets=[]
        for i in range(1, len(sent)): # 1~6
            inputs.append(sent[0:i]) # [[45], [45,139] ...
            targets.append(sent[i])  # [439], [648] ...
        y=np_utils.to_categorical(targets, vocabSize)
        inputSeq=sequence.pad_sequences(inputs, maxlen=maxLen)
        yield(inputSeq,y) # 객체가 리턴 됨
#         print("inputs:",inputs)
#         print("-"*50)
#         print("targets:",targets)
#         print("="*50)

In [51]:
for i, (x,y) in enumerate(genData(doc, maxLen, vocabSize)):
    print(i)
    print("x",x.shape, "\n",x)
    print("y",y.shape, '\n',y)

0
x (187, 187) 
 [[  0   0   0 ...   0   0 102]
 [  0   0   0 ...   0 102  28]
 [  0   0   0 ... 102  28 602]
 ...
 [  0   0 102 ... 647 155   2]
 [  0 102  28 ... 155   2  20]
 [102  28 602 ...   2  20 180]]
y (187, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
1
x (6, 187) 
 [[  0   0   0 ...   0   0  45]
 [  0   0   0 ...   0  45 439]
 [  0   0   0 ...  45 439 648]
 [  0   0   0 ... 439 648 102]
 [  0   0   0 ... 648 102   4]
 [  0   0   0 ... 102   4 649]]
y (6, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
2
x (12, 187) 
 [[  0   0   0 ...   0   0 102]
 [  0   0   0 ...   0 102   1]
 [  0   0   0 ... 102   1 440]
 ...
 [  0   0   0 ...  34 651   4]
 [  0   0   0 ... 651   4  22]
 [  0   0   0 ...   4  22 331]]
y (12, 1165) 
 [[0. 1. 0. ... 0. 0. 0.]
 [0.

107
x (7, 187) 
 [[  0   0   0 ...   0   0  45]
 [  0   0   0 ...   0  45  19]
 [  0   0   0 ...  45  19 838]
 ...
 [  0   0   0 ... 838   4  19]
 [  0   0   0 ...   4  19   2]
 [  0   0   0 ...  19   2 236]]
y (7, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
108
x (19, 187) 
 [[  0   0   0 ...   0   0  19]
 [  0   0   0 ...   0  19  10]
 [  0   0   0 ...  19  10  22]
 ...
 [  0   0   0 ...  40  68  32]
 [  0   0   0 ...  68  32 145]
 [  0   0   0 ...  32 145  11]]
y (19, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
109
x (12, 187) 
 [[  0   0   0 ...   0   0  68]
 [  0   0   0 ...   0  68   1]
 [  0   0   0 ...  68   1  14]
 ...
 [  0   0   0 ...  67 286  47]
 [  0   0   0 ... 286  47  73]
 [  0   0   0 ...  47  73  32]]
y (12, 1165) 
 [[0. 1. 0. ... 

188
x (9, 187) 
 [[  0   0   0 ...   0   0 268]
 [  0   0   0 ...   0 268   1]
 [  0   0   0 ... 268   1  65]
 ...
 [  0   0   0 ... 389   4   9]
 [  0   0   0 ...   4   9  24]
 [  0   0   0 ...   9  24  13]]
y (9, 1165) 
 [[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
189
x (31, 187) 
 [[  0   0   0 ...   0   0  15]
 [  0   0   0 ...   0  15   4]
 [  0   0   0 ...  15   4   9]
 ...
 [  0   0   0 ... 403   3 314]
 [  0   0   0 ...   3 314  12]
 [  0   0   0 ... 314  12  14]]
y (31, 1165) 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
190
x (72, 187) 
 [[  0   0   0 ...   0   0  15]
 [  0   0   0 ...   0  15   4]
 [  0   0   0 ...  15   4 966]
 ...
 [  0   0   0 ... 135   3 314]
 [  0   0   0 ...   3 314  12]
 [  0   0   0 ... 314  12  14]]
y (72, 1165) 
 [[0. 0. 0. ... 

278
x (9, 187) 
 [[   0    0    0 ...    0    0  105]
 [   0    0    0 ...    0  105    1]
 [   0    0    0 ...  105    1 1047]
 ...
 [   0    0    0 ...  301  575    3]
 [   0    0    0 ...  575    3   12]
 [   0    0    0 ...    3   12   14]]
y (9, 1165) 
 [[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
279
x (14, 187) 
 [[   0    0    0 ...    0    0  575]
 [   0    0    0 ...    0  575    1]
 [   0    0    0 ...  575    1  128]
 ...
 [   0    0    0 ... 1048  128   31]
 [   0    0    0 ...  128   31 1049]
 [   0    0    0 ...   31 1049  279]]
y (14, 1165) 
 [[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
280
x (30, 187) 
 [[  0   0   0 ...   0   0  75]
 [  0   0   0 ...   0  75   4]
 [  0   0   0 ...  75   4   9]
 ...
 [  0   0   0 ... 205   3 140]
 [  0   0   0 ...   3

In [45]:
def cf():
    for i in range(3):
        yield i*i
        
print(cf())

<generator object cf at 0x0000017F47915148>


In [44]:
genData(doc, maxLen, vocabSize)

inputs: [[102], [102, 28], [102, 28, 602], [102, 28, 602, 11], [102, 28, 602, 11, 603], [102, 28, 602, 11, 603, 33], [102, 28, 602, 11, 603, 33, 604], [102, 28, 602, 11, 603, 33, 604, 2], [102, 28, 602, 11, 603, 33, 604, 2, 605], [102, 28, 602, 11, 603, 33, 604, 2, 605, 323], [102, 28, 602, 11, 603, 33, 604, 2, 605, 323, 606], [102, 28, 602, 11, 603, 33, 604, 2, 605, 323, 606, 11], [102, 28, 602, 11, 603, 33, 604, 2, 605, 323, 606, 11, 22], [102, 28, 602, 11, 603, 33, 604, 2, 605, 323, 606, 11, 22, 4], [102, 28, 602, 11, 603, 33, 604, 2, 605, 323, 606, 11, 22, 4, 7], [102, 28, 602, 11, 603, 33, 604, 2, 605, 323, 606, 11, 22, 4, 7, 427], [102, 28, 602, 11, 603, 33, 604, 2, 605, 323, 606, 11, 22, 4, 7, 427, 32], [102, 28, 602, 11, 603, 33, 604, 2, 605, 323, 606, 11, 22, 4, 7, 427, 32, 607], [102, 28, 602, 11, 603, 33, 604, 2, 605, 323, 606, 11, 22, 4, 7, 427, 32, 607, 40], [102, 28, 602, 11, 603, 33, 604, 2, 605, 323, 606, 11, 22, 4, 7, 427, 32, 607, 40, 102], [102, 28, 602, 11, 603, 33,

inputs: [[104], [104, 1062], [104, 1062, 3], [104, 1062, 3, 320], [104, 1062, 3, 320, 11], [104, 1062, 3, 320, 11, 72], [104, 1062, 3, 320, 11, 72, 30], [104, 1062, 3, 320, 11, 72, 30, 523]]
--------------------------------------------------
targets: [1062, 3, 320, 11, 72, 30, 523, 524]
inputs: [[45], [45, 86], [45, 86, 86], [45, 86, 86, 10], [45, 86, 86, 10, 302], [45, 86, 86, 10, 302, 46], [45, 86, 86, 10, 302, 46, 3], [45, 86, 86, 10, 302, 46, 3, 1063]]
--------------------------------------------------
targets: [86, 86, 10, 302, 46, 3, 1063, 6]
inputs: [[96], [96, 1], [96, 1, 211], [96, 1, 211, 2], [96, 1, 211, 2, 118], [96, 1, 211, 2, 118, 9], [96, 1, 211, 2, 118, 9, 1], [96, 1, 211, 2, 118, 9, 1, 36], [96, 1, 211, 2, 118, 9, 1, 36, 579], [96, 1, 211, 2, 118, 9, 1, 36, 579, 420]]
--------------------------------------------------
targets: [1, 211, 2, 118, 9, 1, 36, 579, 420, 116]
inputs: [[108], [108, 1]]
--------------------------------------------------
targets: [1, 116]
inputs:

In [59]:
xdata=[]
ydata=[]
for x,y in genData(doc, maxLen, vocabSize):
    xdata.append(x)
    ydata.append(y)
xdata=np.concatenate(xdata)
ydata=np.concatenate(ydata)

In [58]:
# xdata
# 배열 합치기
a=np.array([[1,2],[3,4]])
b=np.array([[5,6],[7,8],[9,10]])
np.concatenate((a,b), axis=0)
np.concatenate((a,b), axis=None)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [61]:
ydata.shape

(6917, 1165)

In [62]:
model=Sequential()
model.add(Embedding(vocabSize, 100, input_length=maxLen)) # 1165,100
model.add(LSTM(100, return_sequences=False))
model.add(Dropout(0,5))
model.add(Dense(vocabSize, activation='softmax'))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [63]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 187, 100)          116500    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1165)              117665    
Total params: 314,565
Trainable params: 314,565
Non-trainable params: 0
_________________________________________________________________


In [64]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [66]:
hist=model.fit(xdata, ydata, epochs=500, batch_size=800)

Epoch 1/500
6917/6917 [==============================] - 16s 2ms/step - loss: 6.1713 - accuracy: 0.0259
Epoch 2/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.7396 - accuracy: 0.0432
Epoch 3/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.6816 - accuracy: 0.0538
Epoch 4/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.6694 - accuracy: 0.0538
Epoch 5/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.6604 - accuracy: 0.0538
Epoch 6/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.6471 - accuracy: 0.0538
Epoch 7/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.6242 - accuracy: 0.0538
Epoch 8/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.5925 - accuracy: 0.0538
Epoch 9/500
6917/6917 [==============================] - 16s 2ms/step - loss: 5.5556 - accuracy: 0.0548
Epoch 10/500
6917/6917 [==============================] - 17s 2m

6917/6917 [==============================] - 17s 2ms/step - loss: 3.0194 - accuracy: 0.3742
Epoch 80/500
6917/6917 [==============================] - 17s 2ms/step - loss: 2.9947 - accuracy: 0.3789
Epoch 81/500
6917/6917 [==============================] - 17s 2ms/step - loss: 2.9649 - accuracy: 0.3846
Epoch 82/500
6917/6917 [==============================] - 17s 2ms/step - loss: 2.9403 - accuracy: 0.3869
Epoch 83/500
6917/6917 [==============================] - 17s 2ms/step - loss: 2.9105 - accuracy: 0.3903
Epoch 84/500
6917/6917 [==============================] - 17s 2ms/step - loss: 2.8879 - accuracy: 0.3966
Epoch 85/500
6917/6917 [==============================] - 17s 2ms/step - loss: 2.8567 - accuracy: 0.4010
Epoch 86/500
6917/6917 [==============================] - 17s 2ms/step - loss: 2.8358 - accuracy: 0.4039
Epoch 87/500
6917/6917 [==============================] - 17s 2ms/step - loss: 2.8071 - accuracy: 0.4106
Epoch 88/500
6917/6917 [==============================] - 17s 2ms/st

6917/6917 [==============================] - 17s 3ms/step - loss: 1.4570 - accuracy: 0.6876
Epoch 157/500
6917/6917 [==============================] - 17s 2ms/step - loss: 1.4352 - accuracy: 0.6925
Epoch 158/500
6917/6917 [==============================] - 17s 3ms/step - loss: 1.4258 - accuracy: 0.6968
Epoch 159/500
6917/6917 [==============================] - 17s 3ms/step - loss: 1.4102 - accuracy: 0.6994
Epoch 160/500
6917/6917 [==============================] - 17s 3ms/step - loss: 1.3959 - accuracy: 0.7042
Epoch 161/500
6917/6917 [==============================] - 18s 3ms/step - loss: 1.3855 - accuracy: 0.7048
Epoch 162/500
6917/6917 [==============================] - 19s 3ms/step - loss: 1.3697 - accuracy: 0.7074
Epoch 163/500
6917/6917 [==============================] - 18s 3ms/step - loss: 1.3555 - accuracy: 0.7103
Epoch 164/500
6917/6917 [==============================] - 18s 3ms/step - loss: 1.3418 - accuracy: 0.7152
Epoch 165/500
6917/6917 [==============================] - 1

6917/6917 [==============================] - 17s 2ms/step - loss: 0.6815 - accuracy: 0.8726
Epoch 234/500
6917/6917 [==============================] - 17s 2ms/step - loss: 0.6764 - accuracy: 0.8736
Epoch 235/500
6917/6917 [==============================] - 17s 2ms/step - loss: 0.6683 - accuracy: 0.8741
Epoch 236/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.6649 - accuracy: 0.8767
Epoch 237/500
6917/6917 [==============================] - 17s 2ms/step - loss: 0.6554 - accuracy: 0.8788
Epoch 238/500
6917/6917 [==============================] - 17s 2ms/step - loss: 0.6484 - accuracy: 0.8803
Epoch 239/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.6468 - accuracy: 0.8786
Epoch 240/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.6369 - accuracy: 0.8815
Epoch 241/500
6917/6917 [==============================] - 17s 2ms/step - loss: 0.6336 - accuracy: 0.8819
Epoch 242/500
6917/6917 [==============================] - 1

6917/6917 [==============================] - 17s 3ms/step - loss: 0.3358 - accuracy: 0.9360
Epoch 311/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.3358 - accuracy: 0.9360
Epoch 312/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.3301 - accuracy: 0.9377
Epoch 313/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.3297 - accuracy: 0.9388
Epoch 314/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.3248 - accuracy: 0.9373
Epoch 315/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.3244 - accuracy: 0.9384
Epoch 316/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.3181 - accuracy: 0.9393
Epoch 317/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.3194 - accuracy: 0.9391
Epoch 318/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.3123 - accuracy: 0.9399
Epoch 319/500
6917/6917 [==============================] - 1

6917/6917 [==============================] - 17s 3ms/step - loss: 0.1911 - accuracy: 0.9540
Epoch 388/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.1916 - accuracy: 0.9529
Epoch 389/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.1886 - accuracy: 0.9539
Epoch 390/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.1886 - accuracy: 0.9545
Epoch 391/500
6917/6917 [==============================] - 18s 3ms/step - loss: 0.1870 - accuracy: 0.9539
Epoch 392/500
6917/6917 [==============================] - 18s 3ms/step - loss: 0.1855 - accuracy: 0.9540
Epoch 393/500
6917/6917 [==============================] - 18s 3ms/step - loss: 0.1851 - accuracy: 0.9545
Epoch 394/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.1841 - accuracy: 0.9543
Epoch 395/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.1831 - accuracy: 0.9542
Epoch 396/500
6917/6917 [==============================] - 1

6917/6917 [==============================] - 17s 3ms/step - loss: 0.1380 - accuracy: 0.9550
Epoch 465/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.1375 - accuracy: 0.9559
Epoch 466/500
6917/6917 [==============================] - 18s 3ms/step - loss: 0.1376 - accuracy: 0.9558
Epoch 467/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.1378 - accuracy: 0.9555
Epoch 468/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.1381 - accuracy: 0.9549
Epoch 469/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.1362 - accuracy: 0.9547
Epoch 470/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.1358 - accuracy: 0.9549
Epoch 471/500
6917/6917 [==============================] - 17s 3ms/step - loss: 0.1359 - accuracy: 0.9565
Epoch 472/500
6917/6917 [==============================] - 18s 3ms/step - loss: 0.1354 - accuracy: 0.9559
Epoch 473/500
6917/6917 [==============================] - 1

In [67]:
import matplotlib.pyplot as plt
plt.plot(hist.history['accuracy'])
plt.show()
model.save('tentGen.hdf5')

<Figure size 640x480 with 1 Axes>